This is for merging all the truth tables together

In [1]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

#sys.path.append("/global/homes/r/rtojeiro/SV/python")
import VI_utils as VI

import matplotlib.pyplot as plt 
import importlib
importlib.reload(VI)
pd.set_option('display.max_rows', 5)

In [2]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/andes/tiles/'
tiles = ['68001']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']

In [3]:
VI_dir = os.environ['HOME']+'/projects/VI_files/Andes_reinspection/LRG/'

In [4]:
vi = VI.read_individual_VI(VI_dir)

Reading all individual VI files in /global/homes/t/tlan/projects/VI_files/Andes_reinspection/LRG/
desi-vi_LRG_reinspection_68001_12_KSD.csv
desi-vi_LRG_reinspection_68001_1_KSD.csv
desi-vi_LRG_reinspection_68001_5_PNT.csv
desi-vi_LRG_reinspection_70002_1_PNT.csv
desi-vi_LRG_reinspection_68001_8_KSD.csv
desi-vi_LRG_reinspection_68001_11_PNT.csv
desi-vi_LRG_reinspection_68001_7_KSD.csv
desi-vi_LRG_reinspection_68001_5_KSD.csv
desi-vi_LRG_reinspection_68001_8_PNT.csv
desi-vi_LRG_reinspection_68001_9_KSD.csv
desi-vi_LRG_reinspection_68001_2_HSH.csv
desi-vi_LRG_reinspection_68001_10_PNT.csv
desi-vi_LRG_reinspection_68001_3_KSD.csv
desi-vi_LRG_reinspection_68001_1_HSH.csv
desi-vi_LRG_reinspection_68001_6_PNT.csv
desi-vi_LRG_reinspection_68001_7_PNT.csv
desi-vi_LRG_reinspection_68001_10_KSD.csv
desi-vi_LRG_reinspection_70002_8_PNT.csv
desi-vi_LRG_reinspection_68001_4_PNT.csv
desi-vi_LRG_reinspection_70002_3_PNT.csv
desi-vi_LRG_reinspection_70002_5_BJW.csv
desi-vi_LRG_reinspection_70002_4_PNT.

In [5]:
vi

,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment
0,3.518641e+16,53122,20200304,70002,0.34.4,-1,-1,GALAXY,1.2321,PNT,1,RS,NaN,GALAXY,Noisy signal Bad fit No line is resolved Possi...
1,3.518641e+16,53122,20200304,70002,0.34.4,-1,-1,GALAXY,0.6524,PNT,4,NaN,0.6524,GALAXY,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,3.519126e+16,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.8300,KSD,4,NaN,0.8300,NaN,NaN
1606,3.519126e+16,55611,20200315,68001,0.34.4,-1,-1,GALAXY,1.0816,KSD,4,NaN,1.0816,NaN,NaN


In [6]:
vi.loc[vi['VI_issue'].isnull(),'VI_issue']='--'
vi['all VI issue'] = vi.groupby('TARGETID')['VI_issue'].transform(lambda x: '|'.join(x))
unique_vi = vi.drop_duplicates(subset=['TARGETID'])

In [7]:
#vi[vi['TARGETID']==35191196661843077]
unique_vi
#vi.loc[vi['VI_comment'].isnull(),'VI_comment']='--'
#vi['all VI comments'] = vi.groupby('TARGETID')['VI_comment'].transform(lambda x: '|'.join(x))


,TARGETID,EXPID,NIGHT,TILEID,Spec_version,Redrock_version,Template_version,Redrock_spectype,Redrock_z,VI_scanner,VI_quality,VI_issue,VI_z,VI_spectype,VI_comment,all VI issue
0,3.518641e+16,53122,20200304,70002,0.34.4,-1,-1,GALAXY,1.2321,PNT,1,RS,NaN,GALAXY,Noisy signal Bad fit No line is resolved Possi...,RS
1,3.518641e+16,53122,20200304,70002,0.34.4,-1,-1,GALAXY,0.6524,PNT,4,--,0.6524,GALAXY,NaN,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,3.518643e+16,53122,20200304,70002,0.34.4,-1,-1,GALAXY,0.9445,PNT,1,RS,NaN,GALAXY,Noisy signal Bad fit No line is resolved Possi...,RS
1515,3.519127e+16,55611,20200315,68001,0.34.4,-1,-1,GALAXY,0.9676,KSD,4,--,0.9676,NaN,NaN,--


In [8]:
filenames = ['truth_table_Andes_reinspection_LRG_68001_20200315_1.csv', 
             'truth_table_Andes_reinspection_LRG_68001_20200315_2.csv', 
             'truth_table_Andes_reinspection_LRG_68001_20200315_3.csv',
             'truth_table_Andes_reinspection_LRG_68001_20200315_4.csv',
            ]

truth_table = pd.read_csv(VI_dir+filenames[0])
for i in range(1,len(filenames)):
    partial_table = pd.read_csv(VI_dir+filenames[i])
    truth_table = truth_table.append(partial_table, ignore_index=True,sort=False)

In [9]:
truth_table=truth_table.merge(unique_vi[['TARGETID', 'all VI issue']], on='TARGETID', how='left')


In [10]:
truth_table

,TARGETID,Redrock_z,Redrock_spectype,best redshift,best spectype,vi_combined_flag,all VI comments,merger comment,N_VI,DELTACHI2,FIBER,all VI issue
0,35191248222424420,0.6752,GALAXY,0.6752,GALAXY,2.000000,--|--|Continuum is shown; but no secure line d...,none,3,25.896473,459,--|--|--
1,35191248226618066,0.9883,GALAXY,0.9883,GALAXY,2.333333,--|--|Continuum detected OII doublet detected ...,none,3,52.546031,470,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
570,35191295236376605,0.8573,GALAXY,0.8573,GALAXY,4.000000,--|--,none,2,99.993594,2661,NaN
571,35191295240570142,0.8363,GALAXY,0.8363,GALAXY,3.500000,--|--,none,2,180.114383,2592,NaN


In [11]:
truth_table = truth_table.rename(columns={"best redshift":"best z", "vi_combined_flag":"best quality"})


In [12]:
truth_table.keys()

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best z', 'best spectype',
       'best quality', 'all VI comments', 'merger comment', 'N_VI',
       'DELTACHI2', 'FIBER', 'all VI issue'],
      dtype='object')

In [13]:
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')

for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

#EXPID = list(set(tf['EXPID']))[0]
#tf = tf[tf['EXPID']==EXPID]
tspec_df = tspec['TARGETID','ZWARN', 'ZERR'].to_pandas()
tf_df = tf['TARGETID','TARGET_RA','TARGET_DEC','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()

unique_tf_df = tf_df.drop_duplicates(subset=['TARGETID'])
unique_tspec_df = tspec_df.drop_duplicates(subset=['TARGETID'])
#print(len(tspec_df))
#print(len(unique_tspec_df))
#print(len(tf_df))
#print(len(unique_tf_df))
#tf_df[tf_df['TARGETID']==35191200671599720]
truth_table = truth_table.merge(unique_tf_df, how='left', on='TARGETID')
truth_table = truth_table.merge(unique_tspec_df, how='left', on='TARGETID')
#tspec_df = tspec['TARGETID','ZWARN', 'ZERR' ].to_pandas()
#tf_df = tf['TARGETID','MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z'].to_pandas()

#tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
#tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

#truth_table = truth_table.merge(tf_df, how='left', on='TargetID')
#truth_table = truth_table.merge(tspec_df, how='left', on='TARGETID')

In [14]:
truth_table.columns

Index(['TARGETID', 'Redrock_z', 'Redrock_spectype', 'best z', 'best spectype',
       'best quality', 'all VI comments', 'merger comment', 'N_VI',
       'DELTACHI2', 'FIBER', 'all VI issue', 'TARGET_RA', 'TARGET_DEC',
       'FLUX_G', 'FLUX_R', 'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R',
       'FIBERFLUX_Z', 'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R',
       'MW_TRANSMISSION_Z', 'ZWARN', 'ZERR'],
      dtype='object')

In [15]:
len(truth_table)

572

In [16]:
truth_table = truth_table[['TARGETID','TARGET_RA','TARGET_DEC','Redrock_z', 'Redrock_spectype', 'best z', 'best spectype',
       'best quality', 'N_VI', 'all VI issue', 'all VI comments', 'merger comment', 'DELTACHI2', 'ZWARN', 'ZERR','FIBER', 
        'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'MW_TRANSMISSION_G',
       'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z']]

In [17]:
save_file = 'Truth_table_Andes_reinspection_LRG_68001_20200315_v1.csv'

In [18]:
truth_table.to_csv(VI_dir+save_file)
